In [8]:
import os
import pandas as pd
from tqdm import tqdm
import json
import matplotlib.pyplot as plt
from scipy.spatial import Voronoi
import numpy as np

# 경로 설정
base_path = 'C:/analyst/StatsBomb Open Data/open-data-master/data'
source_path = os.path.join(base_path, 'refined_three-sixty')
target_path = os.path.join(base_path, 'analysis_events')
three_sixty_path = os.path.join(base_path, 'three-sixty')
os.makedirs(target_path, exist_ok=True)

# 홈/어웨이 팀 추정 함수
def infer_home_away_teams(events_df):
    half_start_teams = events_df.loc[events_df["event_type"] == "Half Start", "team_name"]
    if half_start_teams.empty:
        raise ValueError("Half Start 이벤트가 없어 홈팀 추정이 불가능합니다.")
    home_team = half_start_teams.iloc[0]
    all_teams = events_df["team_name"].dropna().unique().tolist()
    away_team = [team for team in all_teams if team != home_team][0] if len(all_teams) > 1 else None
    return home_team, away_team

# 시퀀스 ID 부여 함수
def assign_sequence_ids(events_df, home_team, away_team):
    ignore_types = {
        "Half Start", "Half End", "Starting XI", "Tactical Shift",
        "Player On", "Player Off", "Injury Stoppage", "Referee Ball Drop", "Bad Behaviour"
    }
    terminate_only_types = {"Substitution"}
    maintain_only_types = {"Duel", "Pressure", "Block", "Dispossessed", "Dribbled Past"}
    set_piece_types = {"Corner", "Free Kick", "Throw In", "Penalty"}

    seq_counters = {home_team: 1, away_team: 1}
    prev_team = None
    prev_time = None
    prev_period = None
    prev_seq_id = None

    sequence_ids = []

    for idx, row in events_df.iterrows():
        team = row["team_name"]
        event_type = row["event_type"]
        time = row["time"]
        period = row["period"]
        tags = row.get("tags", {})

        # Ignore 타입 처리: 그냥 넘어감
        if event_type in ignore_types or pd.isna(team):
            sequence_ids.append(None)
            continue

        # Terminate-only 타입 처리: 시퀀스 넘버만 증가 후 ID는 부여하지 않고 넘어감
        if event_type in terminate_only_types:
            seq_counters[team] += 1
            sequence_ids.append(None)
            continue

        # Maintain-only 타입 처리: 이전 시퀀스 ID 유지
        if event_type in maintain_only_types:
            sequence_ids.append(prev_seq_id)
            continue

        # 나머지 Standard 타입 처리
        # 첫 행이거나, 팀이 바뀌거나, 시간차 15초 이상, 피리어드 변경, 세트피스 발생 시 시퀀스 종료
        possession_change = prev_team is not None and team != prev_team
        period_change = prev_period is not None and period != prev_period
        time_gap = prev_time is not None and abs(time - prev_time) >= 15

        pass_type = tags.get("pass.type.name", "")
        shot_type = tags.get("shot.type.name", "")
        is_set_piece = pass_type in set_piece_types or shot_type in set_piece_types

        sequence_end = possession_change or time_gap or period_change or is_set_piece

        if sequence_end:
            seq_counters[prev_team] += 1

        prefix = "H" if team == home_team else "A"
        current_seq_id = f"{prefix}{seq_counters[team]}"
        sequence_ids.append(current_seq_id)

        # 다음 행 비교를 위해 현재 값 저장
        prev_seq_id = current_seq_id
        prev_team = team
        prev_time = time
        prev_period = period

    events_df["sequence_id"] = sequence_ids
    seq_col = events_df.pop('sequence_id')
    events_df.insert(events_df.columns.get_loc('event_type'), 'sequence_id', seq_col)

    return events_df

# 수비진형라인 부여 함수
def assign_def_line_coordinates(file):
    event_file = os.path.join(source_path, file)
    three_sixty_file = os.path.join(three_sixty_path, file.replace('.pkl', '.json'))

    # 이벤트 데이터 로드
    events_df = pd.read_pickle(event_file)
    # 위치 데이터 로드
    with open(three_sixty_file, 'r', encoding='utf-8') as f:
        three_sixty_data = json.load(f)

    # uuid로 좌표 매핑 생성
    coord_mapping = {}
    keeper_mapping = {}  # [ADD] 상대 GK 좌표 매핑 딕셔너리

    for entry in three_sixty_data:
        uuid = entry.get('event_uuid')
        ff = entry.get('freeze_frame', [])

        opponent_coords = [
            p['location'] for p in ff
            if not p.get('teammate', True) and 'location' in p
        ]
        coord_mapping[uuid] = opponent_coords

        opp_keeper = [  # [ADD] keeper==True 인 상대 GK 좌표
            p['location'] for p in ff
            if not p.get('teammate', True) and p.get('keeper', False) and 'location' in p
        ]
        keeper_mapping[uuid] = opp_keeper[0] if opp_keeper else None  # 없으면 None

    # 최종 수비진형라인 좌표 저장할 열 초기화
    opponent_line_results = []

    # 각 이벤트 행별 수비진형 라인 그룹핑 수행
    for idx, row in events_df.iterrows():
        uuid = row['event_id_full']
        original_coords = coord_mapping.get(uuid, [])
        if not original_coords:
            opponent_line_results.append([])
            continue

        # 미러드 좌표 추가
        all_coords = original_coords.copy()
        for x, y in original_coords:
            all_coords.extend([(x, -y), (x, 136 - y), (-x, y), (208 - x, y)])

        # 보로노이 분할 수행
        vor = Voronoi(all_coords)

        # 수비라인 그룹핑 수행
        remaining_coords = original_coords.copy()
        final_groups = []

        while remaining_coords:
            # 현재 남은 선수 중 x값이 가장 큰 선수의 x좌표 찾기
            x_max_point = max(remaining_coords, key=lambda p: p[0])
            x_line = x_max_point[0]
            current_group = []
            for idx_pt, point in enumerate(remaining_coords):
                region_index = vor.point_region[idx_pt]
                region = vor.regions[region_index]
                if -1 in region or len(region) == 0:
                    continue  # 무한영역 제외
                vertices = vor.vertices[region]
                x_vals = vertices[:, 0]
                if np.min(x_vals) <= x_line <= np.max(x_vals):
                    current_group.append(point)
                # 무한루프 방지 코드 추가!
            if not current_group:
                # 강제로 x_max_point 하나라도 넣어주고 넘어가기
                current_group = [x_max_point]

            # 그룹에 포함된 좌표는 remaining_coords에서 제외
            remaining_coords = [pt for pt in remaining_coords if pt not in current_group]

            # 현재 그룹을 최종 그룹 리스트에 추가
            final_groups.append(current_group)

        # 이번 이벤트의 그룹 결과 저장
        opponent_line_results.append(final_groups)

    # 최종 결과를 opponent_line 열로 저장
    events_df['opponent_line'] = opponent_line_results
    events_df['opponent_keeper'] = events_df['event_id_full'].map(keeper_mapping)

    return events_df

#승패상황 스테이터스 함수
def add_status_statsbomb_simple(
    df,
    home_team,
    away_team,
    *,
    team_col='team_name',      # 이벤트 주체 팀
    type_col='event_type',      # 이벤트 타입: 'Shot' / 'Own Goal For' 등
    tags_col='tags',           # dict-like: tags['shot.outcome_name']=='goal'이면 정상 골
    period_col='period',       # 숫자 피리어드(승부차기=5)
    status_col='status',       # 최종 생성할 상태 열
    insert_pos=6,              # 0-기준 컬럼 인덱스(=7번째) → team_name 뒤, player_name 앞
    debug=False                # True면 임시 열 보존(과정 점검용)
    ):
    """
    득점 인정 규칙(중복 자책 문제 반영):
      - 정상 골: (type=='Shot') AND (tags['shot.outcome_name']=='goal')
      - 자책 득점: (type=='Own Goal For')
      - 그 외는 득점 아님(무시)
    상태(status) 표기:
      - 이벤트 '직전' 스코어 기준으로 +n / 0 / -n (n=골수 차)
      - 승부차기(period==5): status='0'
    위치 규칙:
      - 새 status 컬럼은 0-기준 인덱스 6(=7번째)에 삽입하여
        team_name 바로 뒤, player_name 바로 앞에 오도록 고정합니다.
    """
    import pandas as pd

    # [A] 정상 골/자책 득점 플래그 (대소문자 무시)
    t_lower = df[type_col].astype(str).str.lower()

    # tags에서 'shot.outcome_name'만 확인(스키마 고정)
    def _is_goal_from_tags(tag_obj):
        # dict-like에서 키가 없거나 dict가 아니면 False
        try:
            v = tag_obj.get('shot.outcome_name', '')
        except Exception:
            return False
        return str(v).strip().lower() == 'goal'

    df['__is_shot_goal'] = t_lower.eq('shot') & df[tags_col].apply(_is_goal_from_tags)
    df['__is_og_for']    = t_lower.eq('own goal for')  # 'Own Goal For'도 소문자 비교로 정상 인식

    # [B] 득점팀 결정: 위 두 조건에서만 "이벤트 팀"이 득점
    df['__scoring_team'] = pd.NA
    score_mask = df['__is_shot_goal'] | df['__is_og_for']
    df.loc[score_mask, '__scoring_team'] = df.loc[score_mask, team_col]

    # [C] 누적 스코어(직전 기준): 득점 행 자체 제외를 위해 shift(1)
    df['__home_ev']  = (df['__scoring_team'] == home_team).astype(int)
    df['__away_ev']  = (df['__scoring_team'] == away_team).astype(int)
    df['__home_pre'] = df['__home_ev'].cumsum().shift(1, fill_value=0)  # 이벤트 직전 홈 스코어
    df['__away_pre'] = df['__away_ev'].cumsum().shift(1, fill_value=0)  # 이벤트 직전 원정 스코어

    # [D] 이벤트 주체 관점의 직전 마진: 홈이면 (홈-원정), 원정이면 (원정-홈)
    is_home_evt = df[team_col].eq(home_team)
    team_pre    = df['__home_pre'].where(is_home_evt, df['__away_pre'])
    opp_pre     = df['__away_pre'].where(is_home_evt, df['__home_pre'])
    margin_pre  = (team_pre - opp_pre).astype(int)  # +n/0/-n의 n

    # [E] 표기(+n/0/-n) & 승부차기(5) 마스킹
    status_series = margin_pre.map(lambda m: '0' if m == 0 else (f'+{m}' if m > 0 else f'{m}'))
    # period가 숫자 5이거나 문자열 '5'인 경우 모두 0 처리
    shootout_mask = (df[period_col] == 5) | (df[period_col].astype(str) == '5')
    status_series = status_series.mask(shootout_mask, '0')
    #status_series = margin_pre.astype('int64')
    #status_series = status_series.mask(shootout_mask, 0).astype('int64')

    # [F] 기존 status가 있으면 제거 후, 정확히 0-기준 6 위치에 삽입(= team_name 뒤, player_name 앞)
    if status_col in df.columns:
        df.drop(columns=[status_col], inplace=True)
    df.insert(insert_pos, status_col, status_series)

    # [G] 임시 열 정리 (debug=False일 때 깔끔하게 삭제)
    if not debug:
        df.drop(columns=[
            '__is_shot_goal','__is_og_for','__scoring_team',
            '__home_ev','__away_ev','__home_pre','__away_pre'
        ], inplace=True, errors='ignore')

    return df

#결과지표 xT 함수
def add_xt_delta(df):
    # Karun 12×8 xT 그리드(행=y, 열=x)
    xt = [
       [0.00638303,0.00779616,0.00844854,0.00977659,0.01126267,0.01248344,0.01473596,0.0174506,0.02122129,0.02756312,0.03485072,0.0379259],
       [0.00750072,0.00878589,0.00942382,0.0105949,0.01214719,0.0138454,0.01611813,0.01870347,0.02401521,0.02953272,0.04066992,0.04647721],
       [0.0088799,0.00977745,0.01001304,0.01110462,0.01269174,0.01429128,0.01685596,0.01935132,0.0241224,0.02855202,0.05491138,0.06442595],
       [0.00941056,0.01082722,0.01016549,0.01132376,0.01262646,0.01484598,0.01689528,0.0199707,0.02385149,0.03511326,0.10805102,0.25745362],
       [0.00941056,0.01082722,0.01016549,0.01132376,0.01262646,0.01484598,0.01689528,0.0199707,0.02385149,0.03511326,0.10805102,0.25745362],
       [0.0088799,0.00977745,0.01001304,0.01110462,0.01269174,0.01429128,0.01685596,0.01935132,0.0241224,0.02855202,0.05491138,0.06442595],
       [0.00750072,0.00878589,0.00942382,0.0105949,0.01214719,0.0138454,0.01611813,0.01870347,0.02401521,0.02953272,0.04066992,0.04647721],
       [0.00638303,0.00779616,0.00844854,0.00977659,0.01126267,0.01248344,0.01473596,0.0174506,0.02122129,0.02756312,0.03485072,0.0379259],
    ]

    def _delta(row):
        if row["event_type"] not in ("Pass", "Carry"):
            return 0.0

        # (1) 실패 판정 — Carry 전용: 다음 행 sequence_id의 첫 글자(H/A)가 달라지면 실패
        if row["event_type"] == "Carry":
            try:
                if df["sequence_id"].shift(-1).loc[row.name][0] != row["sequence_id"][0]:
                    return 0.0
            except Exception:
                # 다음 행이 없거나 값이 비어 있으면 실패로 보지 않고 계산 진행
                pass

        # (2) 실패 판정 — Pass 전용: pass.outcome.name 키가 있으면 실패
        if row["event_type"] == "Pass" and "pass.outcome.name" in row["tags"]:
            return 0.0

        # (3) 시작/종료 좌표(둘 다 tags에서 end_location 읽기, 완전 통일)
        sx, sy = row["location"]
        try:
            if row["event_type"] == "Pass":
                ex, ey = row["tags"]["pass.end_location"]
            else:  # Carry
                ex, ey = row["tags"]["carry.end_location"]
        except Exception:
            # end_location이 비어 있으면 해당 이벤트는 기여 0
            return 0.0

        # (4) 104×68 → 12×8 그리드 매핑(인라인 계산, 최소 클램프만)
        ixs = int(sx // (104.0 / len(xt[0]))); iys = int(sy // (68.0 / len(xt)))
        ixe = int(ex // (104.0 / len(xt[0]))); iye = int(ey // (68.0 / len(xt)))
        if ixs < 0: ixs = 0
        if iys < 0: iys = 0
        if ixe < 0: ixe = 0
        if iye < 0: iye = 0
        if ixs >= len(xt[0]): ixs = len(xt[0]) - 1
        if iys >= len(xt):    iys = len(xt)    - 1
        if ixe >= len(xt[0]): ixe = len(xt[0]) - 1
        if iye >= len(xt):    iye = len(xt)    - 1

        return float(xt[iye][ixe] - xt[iys][ixs])

    return pd.DataFrame({"xT_delta": df.apply(_delta, axis=1)}, index=df.index)

#결과지표 seq_hold 함수
def add_seq_hold_sec(df):
    """
    반환: df.index에 정렬된 단일 열 DataFrame({'seq_hold_sec': ...})
    규칙:
      - 기본: (다음 행 time) - (현재 행 time)
      - 단, 다음 행 period가 바뀌면 0초
    """
    import pandas as pd
    next_time   = df["time"].shift(-1)
    next_period = df["period"].shift(-1)
    same_period = df["period"] == next_period
    hold = (next_time - df["time"]).where(same_period, 0)
    return pd.DataFrame({"seq_hold_sec": hold}, index=df.index)

#결과지표 xG 함수
def add_xg_sb(df):
    """
    반환: df.index에 정렬된 단일 열 DataFrame({'xG': ...})
    규칙:
      - event_type == 'Shot' → tags['shot.stats_bomb_xg']
      - 그 외 0.0
    """
    import pandas as pd
    def _xg(row):
        return row["tags"]["shot.statsbomb_xg"] if row["event_type"] == "Shot" else 0.0
    return pd.DataFrame({"xG": df.apply(_xg, axis=1)}, index=df.index)

#5분간 결과지표 함수
def add_next5_labels(df):
    # 결과만 담을 그릇(원본 인덱스 유지)
    labels = pd.DataFrame(index=df.index)
    labels["xT_H_next5"]   = 0.0
    labels["xT_A_next5"]   = 0.0
    labels["hold_H_next5"] = 0.0
    labels["hold_A_next5"] = 0.0
    labels["xG_H_next5"]   = 0.0
    labels["xG_A_next5"]   = 0.0

    # 각 경기·피리어드 안에서만 계산
    for (one_match, one_period), part in df.groupby(["match_id", "period"], sort=False):
        # 시간, 팀(H/A), 값 벡터
        time_array = part["time"].to_numpy()
        is_home    = (part["sequence_id"].str[0] == "H").to_numpy()

        xT_values   = part["xT_delta"].to_numpy()
        hold_values = part["seq_hold_sec"].to_numpy()
        xG_values   = part["xG"].to_numpy()

        # 팀별 누적합(현재 행까지의 합)
        xT_home_cumsum, xT_away_cumsum = [], []
        hold_home_cumsum, hold_away_cumsum = [], []
        xG_home_cumsum, xG_away_cumsum = [], []

        running_xT_home = running_xT_away = 0.0
        running_hold_home = running_hold_away = 0.0
        running_xG_home = running_xG_away = 0.0

        for k in range(len(time_array)):
            if is_home[k]:
                running_xT_home   += xT_values[k]
                running_hold_home += hold_values[k]
                running_xG_home   += xG_values[k]
            else:
                running_xT_away   += xT_values[k]
                running_hold_away += hold_values[k]
                running_xG_away   += xG_values[k]

            xT_home_cumsum.append(running_xT_home)
            xT_away_cumsum.append(running_xT_away)
            hold_home_cumsum.append(running_hold_home)
            hold_away_cumsum.append(running_hold_away)
            xG_home_cumsum.append(running_xG_home)
            xG_away_cumsum.append(running_xG_away)

        # 각 행의 (t, t+300] 구간 끝 위치(해당 period 내부)
        # searchsorted는 t+300 이하의 마지막 인덱스를 찾는 용도(side='right' → -1)
        window_end_index = time_array.searchsorted(time_array + 300.0, side="right") - 1

        # 각 행의 윈도우 합(현재 행은 제외: cumsum[end] - cumsum[i])
        for i, row_index in enumerate(part.index):
            j = window_end_index[i]
            if j >= i:
                labels.at[row_index, "xT_H_next5"]   = xT_home_cumsum[j]   - xT_home_cumsum[i]
                labels.at[row_index, "xT_A_next5"]   = xT_away_cumsum[j]   - xT_away_cumsum[i]
                labels.at[row_index, "hold_H_next5"] = hold_home_cumsum[j] - hold_home_cumsum[i]
                labels.at[row_index, "hold_A_next5"] = hold_away_cumsum[j] - hold_away_cumsum[i]
                labels.at[row_index, "xG_H_next5"]   = xG_home_cumsum[j]   - xG_home_cumsum[i]
                labels.at[row_index, "xG_A_next5"]   = xG_away_cumsum[j]   - xG_away_cumsum[i]
            # j < i 인 경우(남은 시간이 0)에 대해서는 기본 0.0 유지

    return labels[[
        "xT_H_next5","xT_A_next5",
        "hold_H_next5","hold_A_next5",
        "xG_H_next5","xG_A_next5"
    ]]

# 전체 처리 루프
files = [f for f in os.listdir(source_path) if f.endswith('.pkl')]
for file in tqdm(files, desc="Assigning sequence IDs"):
    try:
        filepath = os.path.join(source_path, file)
        events_df = pd.read_pickle(filepath)

        home_team, away_team = infer_home_away_teams(events_df)
        events_df = assign_sequence_ids(events_df, home_team, away_team)

        # 추가 작업: 상대 선수 좌표 추가
        events_df_coords = assign_def_line_coordinates(file)
        events_df['opponent_line'] = events_df_coords['opponent_line']
        insert_at = events_df.columns.get_loc('opponent_line') + 1             # [ADD]
        opk = events_df_coords['opponent_keeper'].map(lambda v: np.nan if v is None else v)
        events_df.insert(insert_at, 'opponent_keeper', opk)  # [ADD]

        # 추가 작업: 스코어 스테이터스 추가
        events_df_status = add_status_statsbomb_simple(events_df, home_team, away_team)
        events_df['status'] = events_df_status['status']
        events_df['status_num'] = pd.to_numeric(events_df['status'], errors='raise').astype('int64')

        # === 지표 3종: 동일 양식(단일 열 DF 반환 → 열 할당) ===
        events_df_xt   = add_xt_delta(events_df)        # xT_delta만 담긴 DF
        events_df_seq  = add_seq_hold_sec(events_df)    # seq_hold_sec만 담긴 DF
        events_df_xg   = add_xg_sb(events_df)           # xG만 담긴 DF

        events_df['xT_delta']    = events_df_xt['xT_delta']
        events_df['seq_hold_sec'] = events_df_seq['seq_hold_sec']
        events_df['xG']          = events_df_xg['xG']

        # === 향후 5분 라벨 6종 추가 ===
        labels_next5 = add_next5_labels(events_df)  # 6개 열만 담긴 DF
        events_df["xT_H_next5"]   = labels_next5["xT_H_next5"]
        events_df["xT_A_next5"]   = labels_next5["xT_A_next5"]
        events_df["hold_H_next5"] = labels_next5["hold_H_next5"]
        events_df["hold_A_next5"] = labels_next5["hold_A_next5"]
        events_df["xG_H_next5"]   = labels_next5["xG_H_next5"]
        events_df["xG_A_next5"]   = labels_next5["xG_A_next5"]

        # === [ADD] 비침습: 홀드 공유율 지표 2개 추가(원본 보존) ===
        events_df["hold_H_next5_share"] = (
            events_df["hold_H_next5"]
            .div(events_df["hold_H_next5"].add(events_df["hold_A_next5"]).replace(0, np.nan))
            .clip(0.0, 1.0))
        events_df["hold_A_next5_share"] = (
            events_df["hold_A_next5"]
            .div(events_df["hold_H_next5"].add(events_df["hold_A_next5"]).replace(0, np.nan))
            .clip(0.0, 1.0))

        # === 저장 ===
        save_path = os.path.join(target_path, file)
        events_df.to_pickle(save_path)

    except Exception as e:
        print(f"⚠️ {file} 처리 중 오류 발생: {e}")

print(f"\n✅ 모든 시퀀스 ID 부여 작업 완료! 저장 경로: {target_path}")


Assigning sequence IDs:  24%|██▍       | 71/295 [07:52<15:57,  4.27s/it]

⚠️ 3835338.pkl 처리 중 오류 발생: Expecting value: line 181321 column 20 (char 5193728)


Assigning sequence IDs:  25%|██▌       | 75/295 [08:10<14:30,  3.96s/it]

⚠️ 3835342.pkl 처리 중 오류 발생: Expecting ',' delimiter: line 171856 column 109 (char 4882432)


Assigning sequence IDs:  27%|██▋       | 80/295 [08:40<16:47,  4.69s/it]

⚠️ 3845506.pkl 처리 중 오류 발생: Expecting ',' delimiter: line 92794 column 3 (char 2637824)


Assigning sequence IDs: 100%|██████████| 295/295 [32:58<00:00,  6.71s/it]


✅ 모든 시퀀스 ID 부여 작업 완료! 저장 경로: C:/analyst/StatsBomb Open Data/open-data-master/data\analysis_events


In [9]:
import pandas as pd
import os
import random

# pandas 출력 옵션 설정 (전체 열 표시)
pd.options.display.expand_frame_repr = False
pd.set_option('display.max_columns', None)
pd.set_option("display.max_colwidth", None)
pd.set_option('display.max_rows', None)
pd.set_option("display.width", 2000)

# 데이터 경로 설정
data_path = 'C:/analyst/StatsBomb Open Data/open-data-master/data/analysis_events'

# 파일 목록 생성
files = [os.path.join(data_path, file) for file in os.listdir(data_path) if file.endswith('.pkl')]

# 랜덤 파일 선택
sample_file = files[0] if len(files) > 71 else random.choice(files)

# 선택한 파일 로딩 및 출력
df = pd.read_pickle(sample_file)
#print(df.drop(columns=['event_id_full']).iloc[30:130].to_string(index=False))
df.drop(columns=['event_id_full']).iloc[30:130]

# 오운골 유형만 필터링하여 확인 (type_name 기준)
#og_df = df[df['event_type'].astype(str).str.lower().isin(['own goal for', 'own goal against'])]
# 노트북/인터랙티브: 표로 바로 확인
df.loc[910:940]
#df[df['event_type']=='Shot'].to_string
#print(df[df['event_type'] == 'Ball Recovery'].to_string(index=False))


,match_id,event_id,event_id_full,period,time,team_name,status,player_name,player_position,play_pattern,sequence_id,event_type,tags,location,opponent_line,opponent_keeper,status_num,xT_delta,seq_hold_sec,xG,xT_H_next5,xT_A_next5,hold_H_next5,hold_A_next5,xG_H_next5,xG_A_next5,hold_H_next5_share,hold_A_next5_share
910,3788741,4603,46035630-1648-46b6-915d-b33edf63a029,1,1211.328,Italy,0,Ciro Immobile,Center Forward,From Throw In,A40,Ball Recovery,"{'timestamp': '00:20:11.328', 'player.name': 'Ciro Immobile'}","[68.64, 40.629999999999995]","[[[89.74224675275289, 42.199197813366865], [90.68737778224379, 61.8115689827168], [90.76094784978311, 30.869227560551728], [94.35019811039486, 49.29091826110888]], [[78.42369366067231, 47.468954725187174], [82.41714131223122, 31.3990920795342], [85.09970066632943, 42.29200218043985]], [[70.9692141528072, 42.501594179479035], [76.5803544213747, 47.55252044744861]]]",NaN,0,0.000000,0.000,0.000000,0.027207,0.197662,163.255,137.257,0.0,0.176041,0.543256,0.456744
911,3788741,4c2a,4c2a1582-fe74-4053-8bf8-42e96586fda6,1,1211.328,Italy,0,Ciro Immobile,Center Forward,From Throw In,A40,Carry,"{'timestamp': '00:20:11.328', 'player.name': 'Ciro Immobile', 'under_pressure': True, 'carry.end_location': [79.38666666666667, 36.04]}","[68.64, 40.629999999999995]","[[[89.74224675275289, 42.199197813366865], [90.68737778224379, 61.8115689827168], [90.76094784978311, 30.869227560551728], [94.35019811039486, 49.29091826110888]], [[78.42369366067231, 47.468954725187174], [82.41714131223122, 31.3990920795342], [85.09970066632943, 42.29200218043985]], [[70.9692141528072, 42.501594179479035], [76.5803544213747, 47.55252044744861]]]",NaN,0,0.015143,1.416,0.000000,0.027207,0.182519,163.255,135.841,0.0,0.176041,0.545828,0.454172
912,3788741,c946,c946dc66-4adf-49d6-a5fe-e5f21ffe3bb3,1,1212.744,Turkey,0,Hakan Çalhanoğlu,Left Midfield,From Throw In,A40,Pressure,"{'timestamp': '00:20:12.744', 'player.name': 'Hakan Çalhanoğlu'}","[31.373333333333335, 26.945]","[[[31.857360881989536, 7.154220086351025], [35.82856712422743, 27.09078094575662], [42.786870281998, 56.277604692569184], [43.03014745450222, 45.60395689575147], [47.97097168472575, 36.39333137612485]], [[26.34406751189374, 49.82657026414246], [34.415099639953446, 33.10631892744562]]]",NaN,0,0.000000,0.496,0.000000,0.027207,0.182519,164.359,135.345,0.0,0.176041,0.548404,0.451596
913,3788741,e3f5,e3f55924-9356-4648-8f0c-d7705de39c14,1,1213.240,Turkey,0,Okay Yokuşlu,Center Defensive Midfield,From Throw In,A40,Pressure,"{'timestamp': '00:20:13.240', 'player.name': 'Okay Yokuşlu'}","[25.133333333333333, 28.560000000000002]","[[[25.01146051178978, 9.383845801547999], [27.429672286593238, 34.92090490264648], [30.50037638988612, 27.4655012131215], [38.37111694636363, 48.19106441953011], [38.82347410815255, 62.23377683904094]], [[22.034436941317736, 51.67586125176791]]]",NaN,0,0.000000,0.968,0.000000,0.029361,0.182519,165.209,134.377,0.0,0.176041,0.551458,0.448542
914,3788741,c815,c8155381-fa91-4685-8c40-397d5d828365,1,1214.208,Turkey,0,Okay Yokuşlu,Center Defensive Midfield,From Throw In,A40,Dribbled Past,"{'timestamp': '00:20:14.208', 'player.name': 'Okay Yokuşlu'}","[24.7, 32.045]","[[[77.33615436353381, 16.082799166028394], [85.12463905332426, 51.159347656720385], [91.5999984741211, 42.400001525878906], [91.81114310057151, 66.87932002398794], [92.30870366809278, 29.885058635944]], [[78.48664747523054, 31.641827220698097]]]",NaN,0,0.000000,0.000,0.000000,0.029361,0.182519,166.572,134.377,0.0,0.176041,0.553489,0.446511
915,3788741,cde7,cde7f927-f0d6-4f5b-b04c-575d603b9f08,1,1214.208,Italy,0,Ciro Immobile,Center Forward,From Throw In,A40,Dribble,"{'timestamp': '00:20:14.208', 'player.name': 'Ciro Immobile', 'under_pressure': True, 'dribble.outcome.name': 'Complete'}","[79.38666666666667, 36.04]","[[[88.55117224177027, 21.853024264712033], [97.5774797342709, 28.43948273690043], [97.95682805662615, 51.84959233621412], [99.11374145946641, 34.05397128833111], [100.59184561542237, 4